In [78]:
import pandas as pd
import numpy as np
from datetime import datetime
customers=pd.read_csv("customers.csv")
sales=pd.read_csv("transactions.csv")
products=pd.read_csv("products.csv")

# Merge all the .csv files before data cleaning

In [79]:
product_sales_merged = pd.merge(sales,products, on='id_prod', how='left')
customer_product_sales = pd.merge(product_sales_merged, customers, on='client_id', how='left')
customer_product_sales

id_prod                        date session_id client_id  price  categ  \
0       0_1483  2021-04-10 18:37:28.723910    s_18746    c_4450   4.99    0.0   
1        2_226  2022-02-03 01:55:53.276402   s_159142     c_277  65.75    2.0   
2        1_374  2021-09-23 15:13:46.938559    s_94290    c_4270  10.71    1.0   
3       0_2186  2021-10-17 03:27:18.783634   s_105936    c_4597   4.20    0.0   
4       0_1351  2021-07-17 20:34:25.800563    s_63642    c_1242   8.99    0.0   
...        ...                         ...        ...       ...    ...    ...   
337011   1_671  2021-05-28 12:35:46.214839    s_40720    c_3454  31.99    1.0   
337012   0_759  2021-06-19 00:19:23.917703    s_50568    c_6268  22.99    0.0   
337013  0_1256  2021-03-16 17:31:59.442007     s_7219    c_4137  11.03    0.0   
337014   2_227  2021-10-30 16:50:15.997750   s_112349       c_5  50.99    2.0   
337015  0_1417  2021-06-26 14:38:19.732946    s_54117    c_6714  17.99    0.0   

       sex  birth  
0        f   1977  
1        f   2000  
2        f   1979  
3        m   1963  
4        f   1980  
...     ..    ...  
337011   m   1969  
337012   m   1991  
337013   f   1968  
337014   f   1994  
337015   f   1968  

[337016 rows x 8 columns]

# Explore the merged dataframe

In [80]:
# earlier examinations of each dataframe revealed: 
# 1. no null value in each dataframe, customers, sales & products.
# 2. 200 duplicate 'session_id'='s_0' with the 'prod_id'='T_0' and 'price'='-1.00' from tests.
# 3. however, merging all 3 dataframes resulted in 103 null values in 'price' and 'categ' columns
# 4. we will be removing this 303 duplicate and null rows.
customer_product_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 337016 entries, 0 to 337015
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   id_prod     337016 non-null  object 
 1   date        337016 non-null  object 
 2   session_id  337016 non-null  object 
 3   client_id   337016 non-null  object 
 4   price       336913 non-null  float64
 5   categ       336913 non-null  float64
 6   sex         337016 non-null  object 
 7   birth       337016 non-null  int64  
dtypes: float64(2), int64(1), object(5)
memory usage: 20.6+ MB


In [81]:
cols = customer_product_sales.columns
cols

Index(['id_prod', 'date', 'session_id', 'client_id', 'price', 'categ', 'sex',
       'birth'],
      dtype='object')

In [82]:
# rename the merged dataframe
new_names = { 'id_prod':'product_id',
             'categ':'category',
             'client_id':'customer_id',
             'birth': 'birth_year'
    }
customer_product_sales.rename(columns=new_names, inplace=True)
customer_product_sales.head()

product_id                        date session_id customer_id  price  \
0     0_1483  2021-04-10 18:37:28.723910    s_18746      c_4450   4.99   
1      2_226  2022-02-03 01:55:53.276402   s_159142       c_277  65.75   
2      1_374  2021-09-23 15:13:46.938559    s_94290      c_4270  10.71   
3     0_2186  2021-10-17 03:27:18.783634   s_105936      c_4597   4.20   
4     0_1351  2021-07-17 20:34:25.800563    s_63642      c_1242   8.99   

   category sex  birth_year  
0       0.0   f        1977  
1       2.0   f        2000  
2       1.0   f        1979  
3       0.0   m        1963  
4       0.0   f        1980

# Define the functions for the dataframe pipe.

In [83]:

# dropping null rows
def drop_nulls(customer_product_sales, cols):
    customer_product_sales.dropna(subset = cols, inplace=True)
    return customer_product_sales
# dropping duplicated rows
def drop_duplicates(customer_product_sales, cols):
    customer_product_sales.duplicated(subset = cols, keep=False)
    return customer_product_sales




# Apply the functions for cleaning the dataframe.

In [84]:
# List and Pipe
null_cols = ['price','category']
duplicates_cols = ['session_id']

id_clean = (
    customer_product_sales
      .pipe(drop_nulls,null_cols)
      .pipe(drop_duplicates,duplicates_cols)
    
)

## Deleting rows with price equals to -1.00. Now we have a dataframe with equal non null values (336713) for all columns. 
id_clean = id_clean.loc[id_clean["price"]>-1]

id_clean.head()

product_id                        date session_id customer_id  price  \
0     0_1483  2021-04-10 18:37:28.723910    s_18746      c_4450   4.99   
1      2_226  2022-02-03 01:55:53.276402   s_159142       c_277  65.75   
2      1_374  2021-09-23 15:13:46.938559    s_94290      c_4270  10.71   
3     0_2186  2021-10-17 03:27:18.783634   s_105936      c_4597   4.20   
4     0_1351  2021-07-17 20:34:25.800563    s_63642      c_1242   8.99   

   category sex  birth_year  
0       0.0   f        1977  
1       2.0   f        2000  
2       1.0   f        1979  
3       0.0   m        1963  
4       0.0   f        1980

In [85]:
id_clean.describe()

price       category     birth_year
count  336713.000000  336713.000000  336713.000000
mean       17.215189       0.430156    1977.823568
std        17.855445       0.591082      13.524433
min         0.620000       0.000000    1929.000000
25%         8.610000       0.000000    1971.000000
50%        13.900000       0.000000    1980.000000
75%        18.990000       1.000000    1987.000000
max       300.000000       2.000000    2004.000000

# Adding the 'age' column to cleaned dataframe

In [86]:
current_year = datetime.now().year
id_clean['age'] = current_year - id_clean['birth_year']
id_clean

C:\Users\dinad\AppData\Local\Temp\ipykernel_19348\2174264009.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  id_clean['age'] = current_year - id_clean['birth_year']


product_id                        date session_id customer_id  price  \
0          0_1483  2021-04-10 18:37:28.723910    s_18746      c_4450   4.99   
1           2_226  2022-02-03 01:55:53.276402   s_159142       c_277  65.75   
2           1_374  2021-09-23 15:13:46.938559    s_94290      c_4270  10.71   
3          0_2186  2021-10-17 03:27:18.783634   s_105936      c_4597   4.20   
4          0_1351  2021-07-17 20:34:25.800563    s_63642      c_1242   8.99   
...           ...                         ...        ...         ...    ...   
337011      1_671  2021-05-28 12:35:46.214839    s_40720      c_3454  31.99   
337012      0_759  2021-06-19 00:19:23.917703    s_50568      c_6268  22.99   
337013     0_1256  2021-03-16 17:31:59.442007     s_7219      c_4137  11.03   
337014      2_227  2021-10-30 16:50:15.997750   s_112349         c_5  50.99   
337015     0_1417  2021-06-26 14:38:19.732946    s_54117      c_6714  17.99   

        category sex  birth_year  age  
0            0.0   f        1977   47  
1            2.0   f        2000   24  
2            1.0   f        1979   45  
3            0.0   m        1963   61  
4            0.0   f        1980   44  
...          ...  ..         ...  ...  
337011       1.0   m        1969   55  
337012       0.0   m        1991   33  
337013       0.0   f        1968   56  
337014       2.0   f        1994   30  
337015       0.0   f        1968   56  

[336713 rows x 9 columns]

# Saving dataframe as .csv

In [87]:
id_clean.to_csv('company_sales.csv', index=False) 

In [88]:
company_sales_clean = pd.read_csv('company_sales.csv')
company_sales_clean.describe()

price       category     birth_year            age
count  336713.000000  336713.000000  336713.000000  336713.000000
mean       17.215189       0.430156    1977.823568      46.176432
std        17.855445       0.591082      13.524433      13.524433
min         0.620000       0.000000    1929.000000      20.000000
25%         8.610000       0.000000    1971.000000      37.000000
50%        13.900000       0.000000    1980.000000      44.000000
75%        18.990000       1.000000    1987.000000      53.000000
max       300.000000       2.000000    2004.000000      95.000000

In [89]:
company_sales_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336713 entries, 0 to 336712
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   product_id   336713 non-null  object 
 1   date         336713 non-null  object 
 2   session_id   336713 non-null  object 
 3   customer_id  336713 non-null  object 
 4   price        336713 non-null  float64
 5   category     336713 non-null  float64
 6   sex          336713 non-null  object 
 7   birth_year   336713 non-null  int64  
 8   age          336713 non-null  int64  
dtypes: float64(2), int64(2), object(5)
memory usage: 23.1+ MB


# Now we have a new cleaned dataframe to perform analysis.